```python
Statistic : 
- 관찰된 빈도와 기대 빈도 간의 차이를 나타내는 값
- 두 변수 간의 관계가 강할수록 큰 값을 나타냄
- 검정 통계량이 클수록 귀무가설을 기각할 확률이 커짐

p-value : 
- 귀무가설이 참일 경우, 극단값을 얻을 확률
- p-value가 작을 수록 귀무가설을 기각할 확률이 높아짐
```

## 가설 검정
#### H0 :  X에 따라 지연 여부에 차이가 없다
#### H1 : X에 따라 지연 여부에 차이가 있다
- 가설1. 출발 공항에 따라 지연 여부에 차이가 없다
- 가설2. 도착 공항에 따라 지연 여부에 차이가 없다
- 가설3. 출발 시간에 따라 지연 여부에 차이가 없다
- 가설4. 도착 시간에 따라 지연 여부에 차이가 없다
- 가설5. 출발 주에 따라 지연 여부에 차이가 없다
- 가설6. 도착 주에 따라 지연 여부에 차이가 없다
- 가설7. 항공사에 따라 지연 여부에 차이가 없다
- 가설8. Carrier_Code에 따라 지연 여부에 차이가 없다
- 가설9. Tail_Number에 따라 지연 여부에 차이가 없다
- 가설10. 달에 따라 지연 여부에 차이가 없다
- 가설11. 일에 따라 지연 여부에 차이가 없다

In [2]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import warnings
import scipy.stats as stats
warnings.filterwarnings('ignore')

In [3]:
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 100)

In [4]:
df = pd.read_csv('train.csv')
df[:5]

,ID,Month,Day_of_Month,Estimated_Departure_Time,Estimated_Arrival_Time,Cancelled,Diverted,Origin_Airport,Origin_Airport_ID,Origin_State,Destination_Airport,Destination_Airport_ID,Destination_State,Distance,Airline,Carrier_Code(IATA),Carrier_ID(DOT),Tail_Number,Delay
0,TRAIN_000000,4,15,NaN,NaN,0,0,OKC,13851,Oklahoma,HOU,12191,Texas,419.0,Southwest Airlines Co.,WN,19393.0,N7858A,NaN
1,TRAIN_000001,8,15,740.0,1024.0,0,0,ORD,13930,Illinois,SLC,14869,Utah,1250.0,SkyWest Airlines Inc.,UA,20304.0,N125SY,NaN
2,TRAIN_000002,9,6,1610.0,1805.0,0,0,CLT,11057,North Carolina,LGA,12953,New York,544.0,American Airlines Inc.,AA,19805.0,N103US,NaN
3,TRAIN_000003,7,10,905.0,1735.0,0,0,LAX,12892,California,EWR,11618,New Jersey,2454.0,United Air Lines Inc.,UA,NaN,N595UA,NaN
4,TRAIN_000004,1,11,900.0,1019.0,0,0,SFO,14771,California,ACV,10157,California,250.0,SkyWest Airlines Inc.,UA,20304.0,N161SY,NaN


In [9]:
df['Estimated_Departure_Time_Hour'] = df['Estimated_Departure_Time'].astype(str).apply(lambda x : x.split('.')[0].zfill(4)[:-2])
df['Estimated_Departure_Time_Hour'] = df['Estimated_Departure_Time_Hour'].replace('0n', np.nan).astype(str)
df['Estimated_Arrival_Time_Hour'] = df['Estimated_Arrival_Time'].astype(str).apply(lambda x : x.split('.')[0].zfill(4)[:-2])
df['Estimated_Arrival_Time_Hour'] = df['Estimated_Arrival_Time_Hour'].replace('0n', np.nan).astype(str)

In [10]:
df.columns

Index(['ID', 'Month', 'Day_of_Month', 'Estimated_Departure_Time',
       'Estimated_Arrival_Time', 'Cancelled', 'Diverted', 'Origin_Airport',
       'Origin_Airport_ID', 'Origin_State', 'Destination_Airport',
       'Destination_Airport_ID', 'Destination_State', 'Distance', 'Airline',
       'Carrier_Code(IATA)', 'Carrier_ID(DOT)', 'Tail_Number', 'Delay',
       'Estimated_Departure_Time_Hour', 'Estimated_Arrival_Time_Hour'],
      dtype='object')

In [11]:
cols = ['Origin_Airport', 'Destination_Airport', 'Estimated_Departure_Time_Hour', 'Estimated_Arrival_Time_Hour',
       'Origin_State', 'Destination_State', 'Airline', 'Carrier_Code(IATA)', 'Tail_Number', 'Month', 'Day_of_Month']

In [17]:
# p-value 담을 데이터 프레임 생성
pvalues = pd.DataFrame()

for col in cols : 
    table = pd.crosstab(df[col], df['Delay'])
    _, p, _, _ = stats.chi2_contingency(table)
    pvalues = pd.concat([pvalues, pd.Series(p)], axis=1)

pvalues.columns = cols
pvalues

,Origin_Airport,Destination_Airport,Estimated_Departure_Time_Hour,Estimated_Arrival_Time_Hour,Origin_State,Destination_State,Airline,Carrier_Code(IATA),Tail_Number,Month,Day_of_Month
0,8.865141e-232,4.386465e-212,0.0,0.0,9.387353e-186,2.111630e-204,2.069111e-274,5.173332e-241,6.253349e-87,1.898826e-222,1.254996e-56


In [19]:
# p-value < 0.05 여부 확인
pvalues.T < 0.05 

,0
Origin_Airport,True
Destination_Airport,True
Estimated_Departure_Time_Hour,True
Estimated_Arrival_Time_Hour,True
Origin_State,True
Destination_State,True
Airline,True
Carrier_Code(IATA),True
Tail_Number,True
Month,True
